In [19]:
import pandas as pd
import csv
import yfinance as yf

#Load Data

#Insider Data Amazon
insider_df = pd.read_csv('C:/Users/trist/code/Data/Finviz_insider/Data/AMZN.csv')

# Stock Data Amazon
ticker_symbol = 'AMZN'
start_date = '2023-01-01'
end_date = '2023-11-30'
amazon_df = yf.download(ticker_symbol, start=start_date, end=end_date)

# insider_df: name='Date', dtype= 'object'
# amazon_df: name='Date', dtype='datetime64[ns]' 


[*********************100%%**********************]  1 of 1 completed


### Todo
dailybasis muss ich urhzeit berücksichtigen

wenn das modell jeden Tag läuft, dann schauen, dass ich das auf Minutenbasis umstelle. yfinance stellt Daten auf minutenbasis bis zu 30 Tage in die Vergangenheit zur Verfügung.

In [27]:
#Enrich df with weekdays and ticker
from datetime import datetime

insider_df['SEC_date'] = pd.to_datetime(insider_df['SEC_date'], format='%b %d %I:%M %p')
insider_df['Date'] = insider_df['SEC_date'].dt.strftime('%d.%m.%Y')
current_year = datetime.now().year
insider_df['Date'] = pd.to_datetime(insider_df['Date'], format='%d.%m.%Y')
insider_df['Date'] = insider_df['Date'].apply(lambda x: x.replace(year=current_year))
insider_df['Weekday_insider_trade'] = insider_df['Date'].dt.day_name()
insider_df['Time'] = insider_df['SEC_date'].dt.strftime('%H:%M')

amazon_df['Weekday_stocks'] = amazon_df.index.day_name()
amazon_df['Ticker'] = ticker_symbol


In [28]:
#Merge
insider_df = insider_df.sort_values('Date')
amazon_df = amazon_df.sort_values('Date')

merged_df = pd.merge_asof(insider_df, amazon_df, on='Date', direction='forward')

In [82]:
#Creating KPIs
small_df = merged_df[['Date', 'Weekday_insider_trade', 'Time', 'insider', 'Open', 'Close', 'Weekday_stocks']]
stock_df_copy = small_df.copy()

# Calculate the daily percentage change using .loc on the copied DataFrame

####Baustelle##############
#stock_df_copy.loc[:, 'DailyChange'] = (stock_df_copy['Close'] - stock_df_copy['Open']) / stock_df_copy['Open'] * 100

### Calculate a 3, 7, 30 day percentage change ###
stock_df_copy['3DayChange'] = 0.0
stock_df_copy['WeekAfterChange'] = 0.0
stock_df_copy['MonthAfterChange'] = 0.0

counter = 0  # Initialize the counter

from pandas.tseries.offsets import BDay
from datetime import timedelta

for index, row in stock_df_copy.iterrows():
    insidertrade_date = row['Date']
    insidertrade_time = row['Time']

    insidertrade_time = datetime.datetime.strptime(insidertrade_time, "%H:%M")
    threshold_time = datetime.datetime.strptime("17:00", "%H:%M")
    
    #falls Formular nach Börsenschluss hochgeladen wird
    if insidertrade_time > threshold_time:
        insidertrade_date = insidertrade_date + BDay(1)

    #Update DailyChange Col
    daily_change = amazon_df[amazon_df.index == insidertrade_date]
    percentage_change = (daily_change['Close'].iloc[-1] - row['Open']) / row['Open'] * 100
    stock_df_copy.at[index, 'DailyChange'] = percentage_change
    
    
    #Update 3DayChange Col
    threedays_after_date = insidertrade_date + BDay(3)  # Use business day offset to add 3 weekdays    
    threedays_after_date = amazon_df[(amazon_df.index >= insidertrade_date) & (amazon_df.index <= threedays_after_date)]

    
    if len(threedays_after_date) < 3: 
        stock_df_copy.at[index, '3DayChange'] = None
    else:
        percentage_change = (threedays_after_date['Close'].iloc[-1] - row['Close']) / row['Close'] * 100
        stock_df_copy.at[index, '3DayChange'] = percentage_change
   

    #Update WeekAfterChange Col
    week_after_date = insidertrade_date + BDay(7)
    week_after_data = amazon_df[(amazon_df.index >= insidertrade_date) & (amazon_df.index <= week_after_date)]
    if len(week_after_data) < 7:
        stock_df_copy.at[index, 'WeekAfterChange'] = None
    else:
        percentage_change = (week_after_data['Close'].iloc[-1] - row['Close']) / row['Close'] * 100
        stock_df_copy.at[index, 'WeekAfterChange'] = percentage_change


    #Update MonthAfterChange Col
    Month_After_Change = insidertrade_date + BDay(30)
    Month_After_Change = amazon_df[(amazon_df.index >= insidertrade_date) & (amazon_df.index <= Month_After_Change)]
    

    if len(Month_After_Change) < 30:
        stock_df_copy.at[index, 'MonthAfterChange'] = None
    else:
        percentage_change = (Month_After_Change['Close'].iloc[-1] - row['Close']) / row['Close'] * 100
        stock_df_copy.at[index, 'MonthAfterChange'] = percentage_change



stock_df_copy.head()

,Date,Weekday_insider_trade,Time,insider,Open,Close,Weekday_stocks,3DayChange,WeekAfterChange,MonthAfterChange,DailyChange
0,2023-03-23,Thursday,16:39,Selipsky Adam,100.430000,98.709999,Thursday,-1.489212,3.748358,5.359134,-1.712637
1,2023-04-05,Wednesday,16:37,Herrington Douglas J,103.910004,101.099998,Wednesday,1.058358,1.394662,14.243325,-2.704268
2,2023-04-06,Thursday,16:42,Selipsky Adam,100.750000,102.059998,Thursday,-2.096805,0.666275,15.765240,1.300246
3,2023-04-06,Thursday,16:42,Selipsky Adam,100.750000,102.059998,Thursday,-2.096805,0.666275,15.765240,1.300246
4,2023-04-21,Friday,18:06,Selipsky Adam,106.099998,106.959999,Friday,2.673897,-3.094613,17.146601,0.103676
